<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
<br>FANAL KRYPTON ANALYSIS</p><br>

In [ ]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
%load_ext memory_profiler
%load_ext line_profiler

import line_profiler
profile = line_profiler.LineProfiler()

In [ ]:
# General importings
import glob
import json

import matplotlib.pyplot        as plt

# Specific IC stuff
import invisible_cities.core.system_of_units  as units

# Specific FANAL stuff
from fanal.fanal_setup       import Setup
from fanal.core.fanal_types  import KrAnalysisParams

# Krypton analysis params

In [ ]:
kr_ifname = "/Users/Javi/Development/nexus-production/data/FLEX100F_M6_O6.EL8bar/kr83.ACTIVE.full/dst/*.h5"
det_name  = "FLEX100F"
verbosity = "WARNING"

In [ ]:
kr_analysis_params = KrAnalysisParams(veto_width          = 20. * units.mm,
                                      tracking_sns_pde    = 1.0,
                                      tracking_mask_att   = 0.0,
                                      tracking_charge_th  = 0.0,
                                      correction_map_type = 'rad'
                                     )

# ANALYSIS functions

In [ ]:
import pandas as pd
import numpy  as np

from typing import Tuple

from invisible_cities.core.core_functions     import in_range

from fanal.core.detectors      import Detector
from fanal.core.detectors      import S1_WIDTH
from fanal.core.sensors        import get_sensor_response
from fanal.core.sensors        import get_energy_response
from fanal.core.sensors        import get_tracking_response
from fanal.containers.kryptons import Krypton
from fanal.utils.types         import XYZ

In [ ]:
from fanal.analysis.kr_analysis import analyze_kr_event

# Krypton analysis

In [ ]:
from invisible_cities.io.mcinfo_io  import load_mcsensor_response_df
from invisible_cities.io.mcinfo_io  import load_mchits_df

from fanal.containers.kryptons      import KryptonList
from fanal.utils.mc_utils           import get_event_numbers_in_file
from fanal.utils.logger             import get_logger
from fanal.core.detectors           import get_detector

In [ ]:
kr_fnames = sorted(glob.glob(kr_ifname))#[0:5]
detector  = get_detector(det_name)
logger    = get_logger('Fanal', verbosity)

In [ ]:
detector
#detector.get_sensor_types()

In [ ]:
kr_analysis_params

In [ ]:
all_kryptons = KryptonList()

In [ ]:
for ifname in kr_fnames:
    event_ids = get_event_numbers_in_file(ifname)
    print(f'\n*** Processing {ifname}  ({len(event_ids)} events) ...\n')
    
    sns_response = load_mcsensor_response_df(ifname)
    mcHits       = load_mchits_df(ifname)
    
    # Looping through all the events in iFile
    for event_id in event_ids:
        logger.info(f"Analyzing Kr event {event_id}")
        krypton_data = analyze_kr_event(detector,
                                        kr_analysis_params,
                                        event_id,
                                        sns_response.loc[event_id, :],
                                        mcHits      .loc[event_id, :])
        all_kryptons.add(krypton_data)


In [ ]:
all_kryptons_df = all_kryptons.df()

In [ ]:
all_kryptons_df.head()

In [ ]:
from fanal.core.correction_maps import build_correction_map
from fanal.core.correction_maps import correct_s2

In [ ]:
fiduc_selection = ((all_kryptons_df.z_rec   >= detector.active_z_min + kr_analysis_params.veto_width) &
                   (all_kryptons_df.z_rec   <= detector.active_z_max - kr_analysis_params.veto_width) &
                   (all_kryptons_df.rad_rec <= detector.active_rad   - kr_analysis_params.veto_width))

In [ ]:
fiduc_kryptons = all_kryptons_df[fiduc_selection].copy()

In [ ]:
fiduc_kryptons.head()

In [ ]:
corr_map = build_correction_map(fiduc_kryptons, kr_analysis_params.correction_map_type, num_bins=100)

In [ ]:
pes_corr = correct_s2(fiduc_kryptons, corr_map, kr_analysis_params.correction_map_type)

In [ ]:
pes_to_MeV = (kr_energy/units.MeV) / fiduc_kryptons.s2_pes_corr.mean()

In [ ]:
fiduc_kryptons.energy_rec = fiduc_kryptons.s2_pes_corr * pes_to_MeV

# Analysis results

In [ ]:
from fanal.core.fanal_units import Qbb
from fanal.core.fanal_units import kr_energy

from fanal.utils.plot_utils import plot_vertices
from fanal.utils.plot_utils import plot_and_fit

In [ ]:
plt.rcParams["figure.figsize"] = 8, 6
plt.rcParams["font.size"     ] = 14

#### Vertices

In [ ]:
#plot_vertices(all_kryptons_df.rename(columns = {'x_true': 'x', 'y_true': 'y', 'z_true': 'z'}),
#              num_bins=40, extra_size = 50)

#### Positions

In [ ]:
dist_x  = fiduc_kryptons.x_true - fiduc_kryptons.x_rec
dist_y  = fiduc_kryptons.y_true - fiduc_kryptons.y_rec

dist_xy = XYZ(fiduc_kryptons.x_true, fiduc_kryptons.y_true, 0.).distance( \
          XYZ(fiduc_kryptons.x_rec , fiduc_kryptons.y_rec , 0.))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist(dist_x, 50);
ax1.set_title ("Distance X", size=16);
ax1.set_xlabel("mm", size=16);
ax1.set_ylabel("entries", size=16);

energy_sns_pde = 0.4
ax2.hist(dist_y, 50);
ax2.set_title (f"Distance Y", size=16);
ax2.set_xlabel("mm", size=16);
ax2.set_ylabel("entries", size=16);

print("Distance X description:")
print(dist_x.describe())
print("Distance Y description:")
print(dist_y.describe())


In [ ]:
plt.hist(dist_xy, bins = 100);
print("Distance XY description:")
dist_xy.describe()

#### Energy - S1

In [ ]:
fiduc_kryptons.s1_pes.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist(fiduc_kryptons.s1_pes, 50, range=[0, 150]);
ax1.set_title ("S1 (raw)", size=16);
ax1.set_xlabel("pes"     , size=16);
ax1.set_ylabel("entries" , size=16);

ax2.hist2d(fiduc_kryptons.rad_true, fiduc_kryptons.s1_pes, (50, 50), range=[[20, 470], [0, 150]]);
ax2.set_title ("S1 (raw)"   , size=16);
ax2.set_xlabel("Radius (mm)", size=16);
ax2.set_ylabel("pes"        , size=16);

#### Light Collection Efficiency

In [ ]:
# The number of initial scintillation photons for kr is 
# the kr energy / Wsci del xenon
xe_Wsci = 39.2 * units.eV
num_kr_s1_photons = kr_energy / xe_Wsci

# This collection efficiency has been computed asuming that the sesnors used to read it:
# Have a pde = 100 %
# Don't have background
fiduc_kryptons['col_eff'] = fiduc_kryptons.s1_pes / num_kr_s1_photons / units.perCent

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist(fiduc_kryptons.col_eff, 50, range=(0, 15));
ax1.set_title ("Light Collection Efficiency", size=16);
ax1.set_xlabel("Eff (%)", size=16);
ax1.set_ylabel("entries", size=16);

energy_sns_pde = 0.4
ax2.hist(fiduc_kryptons.col_eff * energy_sns_pde, 50, range=(0, 6));
ax2.set_title (f"Light Collection Efficiency (sns pde = {energy_sns_pde})", size=16);
ax2.set_xlabel("Eff (%)", size=16);
ax2.set_ylabel("entries", size=16);

(fiduc_kryptons.col_eff * energy_sns_pde).describe()

#### Energy - S2

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist(fiduc_kryptons.s2_pes, 50, (60000, 100000))
ax1.set_title ("S2 (raw)"   , size=16);
ax1.set_xlabel("pes"     , size=16);
ax1.set_ylabel("entries" , size=16);

ax2.hist2d(fiduc_kryptons.rad_true, fiduc_kryptons.s2_pes, (50, 50), range=[[20, 470], [60000, 100000]])
ax2.set_title ("S2 (raw)"   , size=16);
ax2.set_xlabel("Radius (mm)", size=16);
ax2.set_ylabel("pes"        , size=16);

In [ ]:
mu, sigma, fwhm = plot_and_fit(fiduc_kryptons.s2_pes_corr, title="corrected S2", num_bins=100);

print(f"fwhm at Qbb: {fwhm * (kr_energy / Qbb)**.5 / units.perCent:.3} %")

In [ ]:
mu, sigma, fwhm = plot_and_fit(fiduc_kryptons.energy_rec, title="Energy reconstructed", num_bins=100);

print(f"fwhm at Qbb: {fwhm * (kr_energy / Qbb)**.5 / units.perCent:.3} %")

#### Charge

In [ ]:
print(f"REMEMBER ... Current detector:\n {detector}\n")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist(fiduc_kryptons.q_tot, 50)
#ax1.hist(fiduc_kryptons.q_tot, 50, range=[900, 1300])
ax1.set_title ("Q tot"   , size=16);
ax1.set_xlabel("pes"     , size=16);
ax1.set_ylabel("entries" , size=16);

ax2.hist(fiduc_kryptons.q_max, 50)
ax2.set_title ("Q max"   , size=16);
ax2.set_xlabel("pes"     , size=16);
ax2.set_ylabel("entries" , size=16);

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,7))
ax1.hist2d(fiduc_kryptons.rad_true, fiduc_kryptons.q_tot, (50, 50))
#ax1.hist2d(fiduc_kryptons.rad_true, fiduc_kryptons.q_tot, (50, 50), range=[[20, 470], [900, 1300]])
ax1.set_title ("Q tot"      , size=16);
ax1.set_xlabel("Radius (mm)", size=16);
ax1.set_ylabel("pes"        , size=16);

#ax2.hist2d(fiduc_kryptons.rad_true, fiduc_kryptons.q_max, (50, 50))
ax2.hist2d(fiduc_kryptons.rad_true, fiduc_kryptons.q_max, (50, 50), range=[[20, 470], [0, 300]])
ax2.set_title ("Q max"      , size=16);
ax2.set_xlabel("Radius (mm)", size=16);
ax2.set_ylabel("pes"        , size=16);